In [3]:
"""
Mask R-CNN
Configurations and data loading code for MS COCO.

Copyright (c) 2017 Matterport, Inc.
Licensed under the MIT License (see LICENSE for details)
Written by Waleed Abdulla

------------------------------------------------------------

Usage: import the module (see Jupyter notebooks for examples), or run from
       the command line as such:

    # Train a new model starting from pre-trained COCO weights
    python3 coco.py train --dataset=/path/to/coco/ --model=coco

    # Train a new model starting from ImageNet weights
    python3 coco.py train --dataset=/path/to/coco/ --model=imagenet

    # Continue training a model that you had trained earlier
    python3 coco.py train --dataset=/path/to/coco/ --model=/path/to/weights.h5

    # Continue training the last model you trained
    python3 coco.py train --dataset=/path/to/coco/ --model=last

    # Run COCO evaluatoin on the last model you trained
    python3 coco.py evaluate --dataset=/path/to/coco/ --model=last
"""

import os
import time
import numpy as np
%load_ext autoreload
%autoreload 2
 
# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as maskUtils

from   mrcnn.config import Config
import mrcnn.utils as utils
import mrcnn.model as modellib
import mrcnn.dataset as dataset

# Root directory of the project
ROOT_DIR = os.getcwd()

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from mrcnn.coco import CocoDataset, CocoConfig, evaluate_coco, build_coco_results
import pprint as pp
pp = pp.PrettyPrinter(indent=4)
import argparse

# Parse command line arguments
parser = argparse.ArgumentParser(description='Train Mask R-CNN on MS COCO.')
parser.add_argument("command",
                    metavar="<command>",
                    help="'train' or 'evaluate' on MS COCO")
parser.add_argument('--dataset', required=True,
                    metavar="/path/to/coco/",
                    help='Directory of the MS-COCO dataset')
parser.add_argument('--model', required=True,
                    metavar="/path/to/weights.h5",
                    help="Path to weights .h5 file or 'coco'")
parser.add_argument('--logs', required=False,
                    default=DEFAULT_LOGS_DIR,
                    metavar="/path/to/logs/",
                    help='Logs and checkpoints directory (default=logs/)')
parser.add_argument('--limit', required=False,
                    default=500,
                    metavar="<image count>",
                    help='Images to use for evaluation (defaults=500)')
# args = parser.parse_args()
args = parser.parse_args("train --dataset E:\MLDatasets\coco2014 --model mask_rcnn_coco.h5 --limit 10".split())
# pp.pprint(options)
pp.pprint(args)
print("Model:   ", args.model)
print("Dataset: ", args.dataset)
print("Logs:    ", args.logs)
print("Limit:   ", args.limit)

# Configurations

Namespace(command='train', dataset='E:\\MLDatasets\\coco2014', limit='10', logs='E:\\git_projs\\Mask_RCNN_2\\logs', model='mask_rcnn_coco.h5')
Model:    mask_rcnn_coco.h5
Dataset:  E:\MLDatasets\coco2014
Logs:     E:\git_projs\Mask_RCNN_2\logs
Limit:    10


In [5]:
# Configurations
if args.command == "train":
    config = CocoConfig()
    config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]
else:
    class InferenceConfig(CocoConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
        DETECTION_MIN_CONFIDENCE = 0
    config = InferenceConfig()
config.display()

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
# tf_config = tf.ConfigProto()
# tf_config.gpu_options.per_process_gpu_memory_fraction = 0.55
# set_session(tf.Session(config=tf_config))

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.55)
set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))


Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FCN_INPUT_SHAPE                [1024 1024]
GPU_COUNT                      1
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]
MINI_MASK_SHAPE                (56, 56)
NAME                           coco
NUM_CLASSES                    81
POOL_SIZE                      7
POST_NMS_ROIS_INFERE

In [19]:
# Create model
if args.command == "train":
    print('setup model for Training ')
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
else:
    print('setup model for Inference ')
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)

setup model for Training 
>>> Set_log_dir() -- model dir is  E:\git_projs\Mask_RCNN_2\logs
    set_log_dir: Checkpoint path set to : E:\git_projs\Mask_RCNN_2\logs\coco20180404T1830\mask_rcnn_coco_{epoch:04d}.h5
>>> Generate pyramid anchors 
    Size of anchor array is : (261888, 4)
>>> RPN Outputs  <class 'list'>
      rpn_class_logits_5/concat:0
      rpn_class_5/concat:0
      rpn_bbox_5/concat:0
>>> Proposal Layer init complete. Size of anchors:  (261888, 4)
    layer - Scores  (2, 6000)
    layer - Deltas  (2, 6000, 4)
    layer - Anchors  (2, 6000, 4)
   Proposal layer - type of boxes after slice:  <class 'tensorflow.python.framework.ops.Tensor'>
>>> Detection Target Layer : initialization
>>> Detection Target Layer : call  <class 'list'> 4
     proposals.shape    : (2, ?, ?) <class 'tensorflow.python.framework.ops.Tensor'>
     gt_class_ids.shape : (?, ?) <class 'tensorflow.python.framework.ops.Tensor'>
     gt_bboxes.shape    : (?, ?, 4) <class 'tensorflow.python.framework.ops.T

In [ ]:
# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
layers    = [layer for layer in mdl.layers]          # all layer outputs
len(layers)

In [ ]:
print(' layer {} :  {}'.format(idx,layers[0]))
pp.pprint(layers[0]._inbound_nodes[0].__dict__)
pp.pprint(layers[0].__dict__)
print('-------------------------------------------')

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
for idx,layer in enumerate(layers):
    print('>layer {} : name : {}  type: {}'.format(idx,layer.name,layer))
#     pp.pprint(layer._inbound_nodes[0].__dict__)
    print(">> input_shape {} inbound node {} ".format(layer._inbound_nodes[0].input_shapes,layer._inbound_nodes[0]))
#     print('>> inbound node output_shape {}'.format(layer._inbound_nodes[0].output_shapes))
#     if isinstance("layer", mrcnn.model.BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)

#     pp.pprint(layer.__dict__)
    print("<< outbound shape{} input_shape {}".format(layer._outbound_nodes[0].input_shapes,layer._outbound_nodes[0]))
#     print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    print('-------------------------------------------')
    #layer.name, layer.input._keras_shape, layer.output._keras_shap

In [17]:
import pprint as pp
import keras

# for layer in model.keras_model.layers
#     print layer.name 
mdl = model.keras_model
print(' Total number of layers ', len(model.keras_model.layers), type(mdl))

print('\n Inputs')    
for i, inp in enumerate(mdl.input):
    print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))
    
print('\n Outputs')    
for i, outp in enumerate(mdl.output):
    print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))


# pp = pprint.PrettyPrinter()
for idx,layer in enumerate(mdl.layers): 
    if idx < 232 : 
        continue
    print('#'*100)
    print('#  Layer {} :  Name : {}  Type: {}'.format(idx,layer.name,layer))
    print('#'*100)
    if isinstance(layer, keras.engine.training.Model):
        print('#  MODEL LAYER ')    
#         print('\n Inputs')    
#         for i, inp in enumerate(layer.input):
#             print(' {}   {:35s}  {}  '.format(i, inp.name, inp.shape))

#         print('\n Outputs')    
#         for i, outp in enumerate(layer.output):
#             print(' {}   {:35s}  {}  '.format(i, outp.name, outp.shape))
            
    pp.pprint(layer.__dict__)
    num_inbound  = len(layer._inbound_nodes)
    print('  >>>','-'*85)   
    print('  >>>  Number of inbound nodes ', num_inbound)    
    print('  >>>','-'*85)   
    if num_inbound > 0:

        for i, ib_node in enumerate(layer._inbound_nodes):
            print('\t','-'*70)
            print("\t >> {}  {}  ".format(i, ib_node))            
            for j in ib_node.input_tensors:
                print('\t    incoming tensor(s): {:25s}  shape {}'.format(j.name, j.shape))
            print('\t','-'*70)
                
            pp.pprint(ib_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
    
    

#     if isinstance("layer", model.Keras_BatchNorm):
#         print('Batch Nomraliation layer')
    if 'padding' in layer.__dict__:
        print('\t padding : ',layer.padding)
    if 'filters' in layer.__dict__:
        print('\t Filters : ',layer.filters, ' Kernel Size :', layer.kernel_size, 'Stride:', layer.strides, ' Shape',layer.kernel.shape)
    
    if 'activation' in layer.__dict__:
        print('\t ',layer.activation)
    if 'pool_size' in layer.__dict__:
        print('\t poolsize : ',layer.pool_size,'Stride:', layer.strides)
        
    num_outbound = len(layer._outbound_nodes)
    print('  <<<','-'*85)       
    print('  <<<   Number of outbound nodes ', num_outbound)
    print('  <<<','-'*85)       
#     pp.pprint(layer.__dict__)
    if num_outbound > 0:

        for i, ob_node in enumerate(layer._outbound_nodes):
            print("<< outbound {} input_shape {}".format(i,ob_node.input_shapes))
            for j in ob_node.input_tensors:
                print(' from layer: ', j.name, j.shape)
    #         for j in ob_node.outbound_layer:
    #             print(' to layer: ',j.name)
            pp.pprint(ob_node.__dict__, indent=10, width=140)
    #       print('<< outbound node output_shape {}'.format(layer._outbound_nodes[0].output_shapes))
        
    #layer.name, layer.input._keras_shape, layer.output._keras_shape

 Total number of layers  240 <class 'keras.engine.training.Model'>

 Inputs
 0   input_image_3:0                      (?, 1024, 1024, 3)  
 1   input_image_meta_3:0                 (?, ?)  
 2   input_rpn_match_3:0                  (?, ?, 1)  
 3   input_rpn_bbox_3:0                   (?, ?, 4)  
 4   input_gt_class_ids_3:0               (?, ?)  
 5   input_gt_boxes_3:0                   (?, ?, 4)  
 6   input_gt_masks_3:0                   (?, 56, 56, ?)  

 Outputs
 0   output_rois_3/mul:0                  (2, ?, ?)  
 1   proposal_targets_3/target_class_ids:0  (2, ?)  
 2   proposal_targets_3/target_bbox:0     (2, ?, ?)  
 3   proposal_targets_3/target_mask:0     (2, ?, ?, ?)  
 4   cntxt_layer_4/PyFunc:0               <unknown>  
 5   cntxt_layer_4/PyFunc:1               <unknown>  
 6   cntxt_layer_4/PyFunc:2               <unknown>  
 7   cntxt_layer_4/PyFunc:3               <unknown>  
 8   cntxt_layer_4/PyFunc:4               <unknown>  
 9   cntxt_layer_4/PyFunc:5             